In [1]:
# BLOQUE 1: CONFINUGACIÓN Y ÁLGEBRA LINEAL (Z2)

import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.spatial import Delaunay, Voronoi, voronoi_plot_2d
import math
# Gudhi es opcional si queremos implementar Rips "a mano", 
# pero útil para Alpha complejos complejos.
try:
    import gudhi
except ImportError:
    gudhi = None
from scipy.spatial import Voronoi, voronoi_plot_2d

def smithform(matrix):
    """
    Calcula el rango de una matriz sobre Z_2 usando eliminación Gaussiana.
    Referencia: Clase 9, Forma normal de Smith[cite: 5809].
    """
    M = matrix.copy().astype(int)
    rows, cols = M.shape
    rank = 0
    r, c = 0, 0 
    
    while r < rows and c < cols:
        pivot = np.where(M[r:, c:] == 1)
        if pivot[0].size == 0:
            break
        
        pr, pc = pivot[0][0] + r, pivot[1][0] + c
        
        # Swap filas y columnas para traer el pivote a (r,c)
        M[[r, pr], :] = M[[pr, r], :]
        M[:, [c, pc]] = M[:, [pc, c]]
        
        # Eliminar 1s en la fila r y columna c
        for j in range(cols):
            if j != c and M[r, j] == 1:
                M[:, j] = (M[:, j] + M[:, c]) % 2
        
        for i in range(rows):
            if i != r and M[i, c] == 1:
                M[i, :] = (M[i, :] + M[r, :]) % 2
                
        rank += 1
        r += 1
        c += 1
        
    return rank

In [2]:
# BLOQUE 2: CALSE PRINCIPAL (SimplicialComplex)

class SimplicialComplex:
    """
    Clase para manejar Complejos Simpliciales, Filtraciones y Homología.
    """
    def __init__(self, simplices=None):
        self._simplices = set()
        self._simplex_values = {} # Diccionario para almacenar el valor de filtración

        if simplices:
            self.insert(simplices)

    # --- Gestión de Datos y Filtración ---

    def add_simplex(self, simplex, value=0.0):
        """Añade un símplice y todas sus caras, manteniendo la propiedad de clausura."""
        simplex = frozenset(simplex)
        if not simplex: return

        # Añadimos el símplice y todas sus caras (combinations)
        for r in range(1, len(simplex) + 1):
            for face in combinations(simplex, r):
                face = frozenset(face)
                self._simplices.add(face)
                # Actualizamos el valor de filtración (el menor valor posible)
                if face not in self._simplex_values or value < self._simplex_values[face]:
                    self._simplex_values[face] = float(value)

    def insert(self, simplices_input, value=0.0):
        """Interfaz flexible para insertar listas de símplices o tuplas."""
        for item in simplices_input:
            # Detecta si es formato ((0,1), 0.5) o solo (0,1)
            if isinstance(item, tuple) and len(item) == 2 and isinstance(item[0], (tuple, list)):
                self.add_simplex(item[0], item[1])
            else:
                self.add_simplex(item, value)

        ### Equivale a sublevel en los ejemplos de alpha complejos    def filtration(self, t):
        """Retorna un subcomplejo con símplices nacidos antes del tiempo t."""
        valid_data = [(s, v) for s, v in self._simplex_values.items() if v <= t]
        return SimplicialComplex(valid_data)

    # --- Propiedades Topológicas Básicas (Clase 1, 2 y 3) ---

    @property
    def dimension(self):
        if not self._simplices: return -1
        return max(len(f) for f in self._simplices) - 1

    def n_faces(self, n):
        """Devuelve caras de dimensión n ordenadas."""
        faces = [f for f in self._simplices if len(f) - 1 == n]
        return sorted([tuple(sorted(f)) for f in faces])

    @property
    def face_set(self):
        """Todas las caras ordenadas por dimensión y lexicográficamente."""
        return sorted([tuple(sorted(f)) for f in self._simplices], key=lambda x: (len(x), x))

    @property
    def euler_characteristic(self):
        """Suma alternada de símplices: Σ (-1)^k * s_k [cite: 6765]"""
        return sum((-1)**(len(f)-1) for f in self._simplices)

    def st(self, simplex):
        """Estrella de un símplice: todas las co-caras[cite: 4842]."""
        target = frozenset(simplex)
        return [tuple(sorted(s)) for s in self._simplices if target.issubset(s)]

    def lk(self, simplex):
        """Link de un símplice: frontera de la estrella[cite: 4845]."""
        target = frozenset(simplex)
        link_faces = []
        for s in self._simplices:
            if target.issubset(s):
                diff = s - target
                if diff:
                    link_faces.append(tuple(sorted(diff)))
        return SimplicialComplex(link_faces)

    def connected_components(self):
        """Número de componentes conexas usando Union-Find (Betti_0)[cite: 7230]."""
        vertices = set(v for s in self._simplices for v in s)
        parent = {v: v for v in vertices}

        def find(v):
            if parent[v] != v: parent[v] = find(parent[v])
            return parent[v]

        def union(u, v):
            root_u, root_v = find(u), find(v)
            if root_u != root_v: parent[root_u] = root_v

        for edge in self.n_faces(1):
            union(edge[0], edge[1])
            
        return len({find(v) for v in vertices}) if vertices else 0

    # --- Álgebra Homológica (Clase 7, 8, 9) ---

    def boundary_matrix(self, p):
        """Calcula la matriz de borde M_p: C_p -> C_{p-1}[cite: 5687]."""
        if p <= 0: return np.zeros((0, 0))
        
        row_faces = self.n_faces(p-1) # (p-1)-símplices
        col_faces = self.n_faces(p)   # p-símplices
        
        if not col_faces: return np.zeros((0, 0))
        if not row_faces: return np.zeros((0, len(col_faces))) # Caso borde a 0

        idx_map = {frozenset(f): i for i, f in enumerate(row_faces)}
        M = np.zeros((len(row_faces), len(col_faces)), dtype=int)

        for j, sigma in enumerate(col_faces):
            sigma_set = set(sigma)
            for v in sigma: # Quitamos un vértice para hallar la cara
                face = frozenset(sigma_set - {v})
                if face in idx_map:
                    M[idx_map[face], j] = 1
        return M

    def betti_numbers(self):
        """Calcula Betti usando rangos: b_p = s_p - rank(M_p) - rank(M_{p+1})."""
        dim = self.dimension
        bettis = {}
        ranks = {}

        # Pre-calcular rangos de matrices de borde
        for p in range(1, dim + 2):
            M = self.boundary_matrix(p)
            ranks[p] = smithform(M) if M.size > 0 else 0
        
        # Betti 0: s_0 - rank(M_1)
        s0 = len(self.n_faces(0))
        bettis[0] = s0 - ranks.get(1, 0)

        # Betti p > 0
        for p in range(1, dim + 1):
            sp = len(self.n_faces(p))
            # dim(Z_p) = sp - rank(M_p)
            # dim(B_p) = rank(M_{p+1})
            bettis[p] = sp - ranks.get(p, 0) - ranks.get(p + 1, 0)
            
        return bettis

    # --- Algoritmo Incremental (Clase 9) ---
    
    def incremental_algorithm(self):
        """
        Calcula los números de Betti paso a paso siguiendo la filtración.
        Basado en.
        
        Returns:
            history: Lista de diccionarios con los Betti en cada paso.
        """
        # 1. Ordenar todos los símplices por valor de filtración y dimensión
        # (El orden es crucial para el algoritmo incremental)
        ordered_simplices = sorted(
            self._simplex_values.items(), 
            key=lambda x: (x[1], len(x[0])) # Ordenar por (valor, dimensión)
        )
        
        current_bettis = {i: 0 for i in range(self.dimension + 1)}
        history = []
        
        # Complejo auxiliar para ir construyendo K_i
        K_current = SimplicialComplex()
        
        print(f"{'Simplex':<20} | {'Tipo':<10} | {'Bettis'}")
        print("-" * 50)

        for simplex_set, val in ordered_simplices:
            simplex = tuple(sorted(simplex_set))
            dim = len(simplex) - 1
            
            # Calculamos Betti ANTES de añadir
            betti_old = K_current.betti_numbers().get(dim, 0) if dim >=0 else 0
            if dim > 0:
                 betti_old_prev = K_current.betti_numbers().get(dim-1, 0)
            else:
                 betti_old_prev = 0

            # Añadimos el símplice al complejo parcial
            K_current.add_simplex(simplex, val)
            
            # Calculamos Betti DESPUÉS de añadir
            # NOTA: Para un proyecto real optimizado, usaríamos reducción de matrices
            # incremental. Aquí usamos la comparación de Bettis como "oráculo" 
            # para clasificar el símplice, priorizando claridad sobre O(N).
            betti_new = K_current.betti_numbers()
            
            simplex_type = ""
            
            # Lógica de Clase 9 [cite: 5593, 5594]
            # Si beta_p aumenta -> Positivo (crea ciclo)
            # Si beta_{p-1} disminuye -> Negativo (destruye ciclo/conecta componentes)
            
            if betti_new.get(dim, 0) > betti_old:
                simplex_type = "Positivo (+)"
            elif dim > 0 and betti_new.get(dim-1, 0) < betti_old_prev:
                simplex_type = "Negativo (-)"
            else:
                # Caso dimensión 0 siempre aumenta beta_0 (positivo) a menos que fusione
                if dim == 0: simplex_type = "Positivo (+)" 
            
            current_bettis = betti_new
            print(f"{str(simplex):<20} | {simplex_type:<10} | {current_bettis}")
            history.append({'val': val, 'simplex': simplex, 'bettis': current_bettis.copy()})
            
        return history

In [3]:
# BLOQUE 3: CONSTRUCTORES GEOMÉTRICOS (Alpha Complejos y Vietori - Rips)

def circumradius(points_simplex):
    """Calcula el circunradio de un símplice (arista o triángulo)."""
    pts = np.array(points_simplex)
    # Caso Arista
    if len(pts) == 2:
        return np.linalg.norm(pts[0] - pts[1]) / 2.0
    # Caso Triángulo
    elif len(pts) == 3:
        a = np.linalg.norm(pts[0] - pts[1])
        b = np.linalg.norm(pts[1] - pts[2])
        c = np.linalg.norm(pts[2] - pts[0])
        s = (a + b + c) / 2.0
        area = np.sqrt(max(0, s * (s - a) * (s - b) * (s - c))) # max(0,...) evita errores numéricos
        if area == 0: return float('inf')
        return (a * b * c) / (4.0 * area)
    return 0.0

def calculate_alpha_complex(points):
    """
    Calcula la filtración del Alfa Complejo.
    Añade todos los símplices de Delaunay con sus circunradios.
    La clase SimplicialComplex manejará la consistencia (minimizando valores).
    """
    tri = Delaunay(points)
    alpha_simplices = []
    
    # 1. Vértices (Radio 0)
    for i in range(len(points)):
        alpha_simplices.append(((i,), 0.0))
        
    # 2. Triángulos y sus caras
    # Nota: Insertamos el triángulo con su radio. 
    # Insertamos las aristas con SU propio radio (L/2).
    # Si una arista no es Gabriel, geométricamente su "tiempo de entrada" en Alpha
    # sería el del triángulo, pero al usar add_simplex en la clase,
    # si insertamos arista con L/2 y luego triángulo con R, 
    # el sistema de filtración debe interpretarse correctamente.
    # Para igualar el PDF, usaremos los valores geométricos directos.
    
    for simplex_indices in tri.simplices:
        idx = tuple(sorted(simplex_indices))
        pts = points[list(idx)]
        r_tri = circumradius(pts)
        alpha_simplices.append((idx, r_tri))
        
        # Aristas del triángulo
        for edge_indices in combinations(idx, 2):
            edge = tuple(sorted(edge_indices))
            pts_edge = points[list(edge)]
            r_edge = circumradius(pts_edge)
            alpha_simplices.append((edge, r_edge))

    return SimplicialComplex(alpha_simplices)

def calculate_rips_manual(points, epsilon, max_dim=2):
    """
    Construye Vietoris-Rips calculando distancias y cliques manualmente.
    Útil si no queremos depender de Gudhi.
    Referencia: Clase 4, Complejo de Vietoris-Rips[cite: 4058].
    """
    num_points = len(points)
    complex_data = []
    
    # 1. 0-símplices (Vértices)
    for i in range(num_points):
        complex_data.append(((i,), 0.0))
        
    # 2. 1-símplices (Aristas)
    # Calculamos distancias y añadimos si d <= 2*r (ojo con la def de epsilon)
    # Normalmente epsilon es el diámetro máximo permitido (2r) o el radio.
    # Asumiremos epsilon = longitud máxima de arista.
    
    edges = []
    for i in range(num_points):
        for j in range(i + 1, num_points):
            dist = np.linalg.norm(points[i] - points[j])
            if dist <= epsilon:
                edge = (i, j)
                complex_data.append((edge, dist))
                edges.append(set(edge))
    
    # 3. Expansión a dimensiones superiores (Cliques)
    # Esto es costoso (NP-Hard general), pero aceptable para pocos puntos.
    # Algoritmo simple: buscar triángulos formados por aristas existentes.
    if max_dim >= 2:
        import networkx as nx # Si se permite, facilita mucho buscar cliques
        # Si no, hacemos fuerza bruta para triángulos:
        for i in range(num_points):
            for j in range(i+1, num_points):
                for k in range(j+1, num_points):
                    # Si existen las 3 aristas, añadimos el triángulo
                    if (np.linalg.norm(points[i]-points[j]) <= epsilon and
                        np.linalg.norm(points[j]-points[k]) <= epsilon and
                        np.linalg.norm(points[i]-points[k]) <= epsilon):
                        
                        # El valor de filtración es la arista más larga (diámetro)
                        max_edge = max(
                            np.linalg.norm(points[i]-points[j]),
                            np.linalg.norm(points[j]-points[k]),
                            np.linalg.norm(points[i]-points[k])
                        )
                        complex_data.append(((i,j,k), max_edge))
                        
    return SimplicialComplex(complex_data)

In [4]:
# BLOQUE 4.1: EJECUCIÓN Y ANÁLISIS - Ejemplos básicos y Homología de Anillo

print("\n=== ANÁLISIS DE HOMOLOGÍA: EL ANILLO ===")

# Construimos el anillo manualmente
K_anillo = SimplicialComplex()
simplices_anillo = [
    (0,1,3), (1,3,4), (1,2,4), (2,4,5), (2,0,5), (0,3,5) 
]
K_anillo.insert(simplices_anillo)

print(f"Caras del anillo: {K_anillo.face_set}")

# Calculamos los números de Betti
bettis = K_anillo.betti_numbers() 
print(f"Números de Betti: {bettis}") 

# --- CORRECCIÓN AQUÍ (usar 'bettis' en plural) ---
if bettis.get(1) == 1:
    print(">> Resultado Correcto: Detectado 1 agujero (B1=1).")
else:
    print(f">> Resultado inesperado: Se esperaba B1=1, se obtuvo {bettis.get(1)}")


=== ANÁLISIS DE HOMOLOGÍA: EL ANILLO ===
Caras del anillo: [(0,), (1,), (2,), (3,), (4,), (5,), (0, 1), (0, 2), (0, 3), (0, 5), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5), (0, 1, 3), (0, 2, 5), (0, 3, 5), (1, 2, 4), (1, 3, 4), (2, 4, 5)]
Números de Betti: {0: 1, 1: 1, 2: 0}


NameError: name 'betti' is not defined

In [ ]:
#BLOQUE 4.2: EJECUCIÓN Y ANÁLISIS - Algoritmo Incremental

print("\n=== ALGORITMO INCREMENTAL ===")
# Ejemplo sencillo: Crear un triángulo hueco (ciclo) paso a paso.
sc_inc = SimplicialComplex()
# Filtración:
# t=1: 3 vértices
# t=2: 3 aristas (cierra el ciclo) -> Nace B1
# t=3: 1 triángulo (rellena el ciclo) -> Muere B1
datos = [
    ((0,), 1.0), ((1,), 1.0), ((2,), 1.0), # Vértices
    ((0,1), 2.0), ((1,2), 2.0), ((0,2), 2.0), # Aristas
    ((0,1,2), 3.0) # Triángulo
]
sc_inc.insert(datos)

historia = sc_inc.incremental_algorithm()

In [ ]:
#BLOQUE 4.3: EJECUCIÓN Y ANÁLISIS - Nube de Puntos y Rips

print("\n=== FILTRACIÓN DE VIETORIS-RIPS (NUBE DE PUNTOS) ===")
points = np.array([
    (0.38, 0.46), (0.79, 0.49), (0.56, 0.03), (0.33, 0.71),
    (0.08, 0.22), (0.51, 0.62), (0.33, 0.84), (0.33, 0.45),
    (0.11, 0.66), (0.93, 0.23)
])

# Usamos nuestra implementación manual o Gudhi si prefieres
K_rips = calculate_rips_manual(points, epsilon=0.4, max_dim=2)

print(f"Complejo Rips (epsilon=0.4): {len(K_rips.face_set)} símplices.")
print(f"Betti: {K_rips.betti_numbers()}")

# Visualización (usando tu función plot, ligeramente ajustada)
def plot_complex_2d(points, complex_obj, title):
    plt.figure(figsize=(6,6))
    # Dibujar aristas
    for s in complex_obj.n_faces(1):
        p1, p2 = points[s[0]], points[s[1]]
        plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b-', alpha=0.3)
    # Dibujar triángulos
    for s in complex_obj.n_faces(2):
        pts = points[list(s)]
        plt.fill(pts[:,0], pts[:,1], 'cyan', alpha=0.2)
    # Dibujar puntos
    plt.plot(points[:,0], points[:,1], 'ro')
    plt.title(title)
    plt.show()

plot_complex_2d(points, K_rips, "Vietoris-Rips (r=0.4)")